## Piccard2 Testing Notebook

Google Colab: https://colab.research.google.com/drive/1xjuNG-ebiLlx506hnJM1pvNmSHd3yNmP?usp=sharing

This is a sample notebook that demonstrates usage of the functions in piccard2 using data collected from the cancensus R package.

As you probably know, the piccard package is intended to make it easier to analyze geographically inconsistent census data using a graph data structure. piccard in its current iteration creates the graph network from census data files, and piccard2, which is demonstrated here, applies an existing clustering package (tscluster) to this graph network, which aids analysis and visualizations of the patterns in census data.

In this notebook, we'll go through the clustering process and explore visualizations of the resulting clusters.

First, we'll install all necessary dependencies. piccard and piccard2 will be installed from the new piccard2 git repo, which is necessary to get around some issues in the first iteration of piccard.

In [ ]:
# install piccard and piccard2
import os
import subprocess

# Clone the repo (only if not already cloned)
if not os.path.exists("piccard2"):
    subprocess.run(["git", "clone", "https://github.com/ecorbin567/piccard2.git"])

# Change directory
os.chdir("piccard2/src/piccard")

import piccard as pc
import piccard2 as pc2

# install other dependencies, pip install first if needed
from tscluster.tsplot import tsplot
import geopandas as gpd
import numpy as np
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings('ignore')

Now we'll use geopandas to read the census data files obtained from cancensus. These files contain necessary information about each census tract in the city of Toronto, as well as three variables for each census tract: number of occupied private dwellings, single-detached houses, and apartments in buildings with five or more storeys.

We rename the columns corresponding to these three variables so that they are treated as the same variable across years.

In [ ]:
households_data_2021 = gpd.read_file("https://raw.githubusercontent.com/ecorbin567/piccard2/refs/heads/main/docs/piccard2_testing_data/households_data_2021.geojson")
households_data_2016 = gpd.read_file("https://raw.githubusercontent.com/ecorbin567/piccard2/refs/heads/main/docs/piccard2_testing_data/households_data_2016.geojson")
households_data_2011 = gpd.read_file("https://raw.githubusercontent.com/ecorbin567/piccard2/refs/heads/main/docs/piccard2_testing_data/households_data_2011.geojson")
households_data_2006 = gpd.read_file("https://raw.githubusercontent.com/ecorbin567/piccard2/refs/heads/main/docs/piccard2_testing_data/households_data_2006.geojson")

households_data_2021.rename(columns={'v_CA21_434: Occupied private dwellings by structural type of dwelling data': 'occupied_private_dwellings',
                                     'v_CA21_435: Single-detached house': 'single_detached_house',
                                     'v_CA21_440: Apartment in a building that has five or more storeys': 'apt_five_or_more'}, inplace=True)
households_data_2016.rename(columns={'v_CA16_408: Occupied private dwellings by structural type of dwelling data': 'occupied_private_dwellings',
                                     'v_CA16_409: Single-detached house': 'single_detached_house',
                                     'v_CA16_410: Apartment in a building that has five or more storeys': 'apt_five_or_more'}, inplace=True)
households_data_2011.rename(columns={'v_CA11F_199: Total number of occupied private dwellings by structural type of dwelling': 'occupied_private_dwellings',
                                     'v_CA11F_200: Single-detached house': 'single_detached_house',
                                     'v_CA11F_201: Apartment, building that has five or more storeys': 'apt_five_or_more',}, inplace=True)
households_data_2006.rename(columns={'v_CA06_119: Total number of occupied private dwellings by structural type of dwelling - data': 'occupied_private_dwellings',
                                     'v_CA06_120: Single-detached house': 'single_detached_house',
                                     'v_CA06_124: Apartment, building that has five or more storeys': 'apt_five_or_more',}, inplace=True)

Now we'll create a piccard network table from the data files obtained above. See the piccard documentation for more information about how this is done.

In [ ]:
census_dfs = [households_data_2006, households_data_2011, households_data_2016, households_data_2021]
years = ['2006', '2011', '2016', '2021']

network_table = pc.create_network_table(census_dfs, years, 'GeoUID')
network_table

Next, we'll turn this network table into a 3d numpy array of all possible paths and their corresponding features so we can use it for clustering. In addition to creating the numpy array with clustering_prep, we'll also use tscluster's tsplot package to plot the values for each feature across timesteps. These values will be clustered into groups in the next step.

Note that when specifying the list of columns, you need to specify the column for each individual year in the data. This is because of the way the network table renames columns, but don't worry, every feature is treated the same regardless of year.

In [ ]:
arr, label_dict = pc2.clustering_prep(network_table, 'name', [
    'occupied_private_dwellings_2006', 'single_detached_house_2006', 'apt_five_or_more_2006',
    'occupied_private_dwellings_2011', 'single_detached_house_2011', 'apt_five_or_more_2011',
    'occupied_private_dwellings_2016', 'single_detached_house_2016', 'apt_five_or_more_2016',
    'occupied_private_dwellings_2021', 'single_detached_house_2021', 'apt_five_or_more_2021'])

fig, ax = tsplot.plot(X=arr, label_dict=label_dict)

Before clustering, we will use the elbow function to determine the optimal number of clusters.

In [ ]:
from yellowbrick.cluster import KElbowVisualizer
from sklearn.cluster import KMeans

reshaped_arr = arr.reshape(587*4,3)
reshaped_arr = np.nan_to_num(reshaped_arr, nan=-1000000)

model = KMeans(random_state=4,n_init=10)
visualizer = KElbowVisualizer(model, k=(2,10),timings=False, ax=plt.gca())
visualizer.fit(reshaped_arr)       # Fit data to visualizer
plt.title('Elbow function for Optimal k using Sum of Square Error')
plt.xlabel('k')
plt.ylabel('Sum of Squared Error')
plt.show()

There are lots of ways to customize the cluster function, so feel free to try different clustering schemes, algorithms, or even numbers of clusters if the elbow plot is inconclusive.

Before running cluster, we create a piccard graph and pass it into the function.

In [ ]:
G = pc.create_network(census_dfs, years, 'GeoUID', 0.05)
tsc = pc2.cluster(network_table, G, 'GeoUID', 4, arr=arr, label_dict=label_dict)

As you can see below, the network table now contains new columns with each path's cluster assignment at each timestep, and each node in the graph now contains a new feature: each node's cluster assignment.

In [ ]:
network_table.head(20)

In [ ]:
list(G.nodes(data=True))[:3]

Finally, we'll use piccard2's plot_clusters function, inspired by tsplot, to visualize the cluster assignments. This visualization is not intended for interpreting the results of the clustering or its real-world meaning, but rather for evaluating how well the clustering algorithm performed and what changes might need to be made for a better fit.

Similarly to the cluster function, plot_clusters is very customizable, so feel free to try out visualizing different groups of clusters together!

In [ ]:
pc2.plot_clusters(network_table, tsc)

Now it's time to start interpreting the cluster assignments! parallel_plot shows a Sankey diagram of each path's cluster assignments over time.

In [ ]:
figure = pc2.parallel_plot(network_table,'cluster_assignment', years,height=600)
figure.show()

cluster_count_plot shows just the number of paths assigned to each cluster over time, but it provides a helpful colour-coding of clusters to differentiate them.

In [ ]:
pc2.cluster_count_plot(network_table, 'cluster_assignment', years, x_label='Year', y_label='Number of Census Tracts', figure_size=(10,5), stacked=True)

Finally, clustered_map_plot shows a map of cluster assignments at their real geographic locations at each timestep. 

There are two ways to run this function; you could run the helper function join_geometries yourself and pass in the result (this is recommended if you are using a column id different than the default GeoUID), or you could pass in the network table and geographical data file and have join_geometries run for you.

First method: running join_geometries yourself.

In [ ]:
for year in years:
    network_table[f'cluster_assignment_{year}'] = network_table[f'cluster_assignment_{year}'].astype(str)

# Plot for each year
for year in ['2006', '2011', '2016', '2021']:
    gdf = pc2.join_geometries(f"piccard2_testing_data/households_data_{year}.geojson",network_table, year,"GeoUID","geouid")
    pc2.clustered_map_plot(year, cluster_col_prefix="cluster_assignment", gdf=gdf)

Second method: passing in network table and geographical data file. We used geojson but this works for any file readable by geopandas.

In [ ]:
for year in ['2006', '2011', '2016', '2021']:
    pc2.clustered_map_plot(
        year=year,
        cluster_col_prefix='cluster_assignment',
        geojson_path=f"piccard2_testing_data/households_data_{year}.geojson",
        network_table=network_table
    )